# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [8]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.head(10))
data.info()

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

**Выводы**
* Таблица содержит пропуски в столбцах 'days_employed' - 19351 non-null float64 и total_income - 19351 non-null float64 против 21525 значений во всех остальных столбцах.
* Данные в столбце 'days_employed' - 19351 non-null float64 являются вещественными числами, возможно, полученными арифметическими операциями. Для целей дальнейшей обработки необходимо изменить тип данных на целочисленный.
* В столбце 'education' данные представлены строками в разных регистрах. 
* Столбцы 'education' и  'education_id',вероятно, являются дублирующими.
* Столбцы'family_status' и  'family_status_id', вероятно, являются дублирующими.
* Наименования столбцов имеют приемлемый вид(нет сочетания строчных и прописных букв, "змеиный" стиль, соотносятся с информацией в столбцах по смыслу).
* Столбец 'purpose'  содержит текстовые многословные описания цели кредита, усложняющие работу и добавляющие возможность ошибки, необходимо создать справочник.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**
Одинаковое количество пропусков в 'days_employed' и 'total_income' может быть связано с расчетами показателей или их выгрузкой.

In [10]:
data_isna = data[data['days_employed'].isna() == True]
print(data_isna.head(10))

    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   
65         0            NaN         21   среднее             1   
67         0            NaN         52    высшее             0   
72         1            NaN         32    высшее             0   
82         2            NaN         50    высшее             0   
83         0            NaN         52   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    

**Вывод**
Гипотеза о взаимосвязи пропусков в 'days_employed' и 'total_income' пока подтверждается. Мы видим строки с одновременно пропущенными значениями в обоих столбцах. 
Пропусков данных больше 10%, это слишком большой процент пропусков, чтобы просто выбросить строки с пропусками без потери достоверности данных. 
Проверим, как соотносятся пропуски и типы занятости людей, может быть, мы имеем дело с иждивенцами.

In [11]:
data_isna.groupby('income_type')['income_type'].count().sort_values(ascending = False)

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Сотрудники банка без дохода - явление невозможное. Нужно готовить подстановки значений. Заменим пропуски доходов и стажа медианными значениями по группе.

In [12]:
# вычислим медианный доход по группам:
employe_income = data[data['income_type'] == 'сотрудник']['total_income'].median()
employe_days = abs(data[data['income_type'] == 'сотрудник']['days_employed']).median() # значение берем по модулю, так как в столбце замечены и отрицательные и положительные значения. 
# отрицательные значения могли быть получены как разница с текущей датой или датой подачи заявки на кредит. Экстремальные значения отсекает применение медианы, а не среднего.

companion_income = data[data['income_type'] == 'компаньон']['total_income'].median()
companion_days = abs(data[data['income_type'] == 'компаньон']['days_employed']).median()

retired_income = data[data['income_type'] == 'пенсионер']['total_income'].median()
retired_days = abs(data[data['income_type'] == 'пенсионер']['days_employed']).median()

state_income = data[data['income_type'] == 'госслужащий']['total_income'].median()
state_days = abs(data[data['income_type'] == 'госслужащий']['days_employed']).median()

entrepreneur_income = data[data['income_type'] == 'предприниматель']['total_income'].median()
entrepreneur_days = abs(data[data['income_type'] == 'предприниматель']['days_employed']).median()

#замена значений:

data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
data['days_employed'] = abs(data['days_employed'])
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))
    
# проверим, остались ли пропуски:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> 
<br>То что нужно! Это отличный способ для замены пропусков в данном случае.
</div>

### Замена типа данных

In [13]:
#Изменим тип данных в столбце ['days_employed'] на integer.  Проверяем полученный результат, вызвав метод info()
#Используем метод astype, так как у нас уже вещественные числа float, которые мы хотим превратить в натуральные - integer

data['days_employed'] =abs(data['days_employed'].astype('int'))
data.info()

print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   Среднее             1   
3         3           4124         32   среднее             1   
4    

**Вывод**
Метод изменил тип данных на integer, все значения положительные

In [14]:
data['total_income'] = data['total_income'].astype('int')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**
Метод изменил тип данных на integer 

### Обработка дубликатов

In [15]:
# проверяем данные на наличие явных дубликатов
print(data.duplicated().sum())

54


**Вывод**
В наличии явные дубликаты, 
удаляем методом drop_duplicates 

In [16]:
data = data.drop_duplicates().reset_index(drop=True)
# повторяем проверку данных на наличие явных дубликатов
print(data.duplicated().sum())

0


In [17]:
#традиционно оценим размер таблицы, теперь после удаления  дубликатов:
data.shape[0]

21471

Таблица сократилась, число строк теперь 21471

In [18]:
display(data['education'].value_counts())

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В столбце 'education' были замечены неявные дубликаты, связанные с использованием заглавных букв. Такой тип дубликатов возникает при консолидации данных из разных источников без унификации, либо при ручном вводе данных (без использования "выпадающего" справочника). Устраняем методом str.lower():

In [19]:

data['education'] = data['education'].str.lower()

display(data['education'].value_counts())

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Лемматизация

Так как нам предстоит анализ возвращаемости кредита в зависимости от целей, на которые он получен, посмотрем перечень уникальных целей:
    

In [20]:
display(data['purpose'].value_counts())

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**
Цели в столбце не унифицированы. Одна и та же цель, связанная с жильем может быть: "покупка жилья", "жилье", "покупка жилья для семьи"; или со свадьбой: "свадьба", "на проведение свадьбы", "сыграть свадьбу". Необходима лемматизация для дальнейшей категоризации данных.


Лемматизацию, а не стемминг используем как более продвинутый процесс поиска слов, записанных в разных формах. Лемматизация возвращает слово в словарной форме (для существительных - в именительном падеже) и учитывает его значение, сохраняя смысл

In [21]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
# purpose_lemmas = m.lemmatize(data['purpose'])

def create_category_purpose(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if (('жилье' in lem_purpose) or 
        ('недвижимость' in lem_purpose )):
        return 'недвижимость'
    if 'свадьба' in lem_purpose:
        return 'свадьба'
    if 'образование' in lem_purpose:
        return 'образование'
    return 'прочее' # на случай, если что-то пропустила

data['categorized_purpose'] = data.apply(create_category_purpose, axis=1)




### Категоризация данных

Проверяем категории целей в столбце:

In [22]:
print(data['categorized_purpose'].value_counts())

print(data['categorized_purpose'].value_counts().sum())


недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: categorized_purpose, dtype: int64
21471


**Вывод** 

Все четыре категории целей вошли в итоговый столбец, "прочего" не оказалось. Во всех 21471 строки, оставшиеся после удаления дубликатов есть категория цели. Можно приступать к анализу по категориям и проверкой гипотез.

Категорицацию по остальным сегментам пользователей, в зависимости от направления анализа проведем в соответствующих разделах ниже.

## Шаг 3. Ответьте на вопросы

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Чтобы ответить на этот вопрос, посмотрим, какие варианты "детности" представлены в таблице:

In [23]:
print(data['children'].value_counts())

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Столбец содержит два аномальных значения: -1 и 20. Возможно, вызваны ошибками при вводе. -1 - это, очевидно, 1 ребенок. Исправим эту ошибку:

In [24]:
data['children'] = abs(data['children'])
print(data['children'].value_counts())

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Значение вошло в состав семей с 1 ребенком. Но как расценить семьи с 20 детьми? С одной стороны, логично, что семьи с таким количеством детей обращаются за кредитом, без него справиться, наверное, трудно. Но, с другой стороны, настораживает, что по данным из открытых источников, всего семей , где более 11 детей в России в 2018 году меньше тысячи (https://www.virtualacademy.ru/news/statistika-kolichestva-detei-v-seme-v-rossii/)
Данных о наличии 20 детей на просторах интернета не найдено. Настораживает, что при общем уменьшении числа заемщиков при увеличении числа детей - доля с 20 детьми больше, чем с 4 и 5. Делаем вывод, что это ошибка ввода, на самом деле детей 2 (в реальности можно сделать проверку, попросив кредитный отдел выборочно "поднять" данные заемщиков). Для замены применяем метод replace:

In [25]:
data['children'] = data['children'].replace(20, 2 )
print(data['children'].value_counts())


0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


Разобьем массив по категориям в зависимости от наличия детей:


In [26]:
def create_category_children(row):
    if row['children'] == 0:
        return 'нет детей'
    return 'есть дети'

data['categorized_children'] = data.apply(create_category_children, axis=1)

print(data['categorized_children'].value_counts())

нет детей    14107
есть дети     7364
Name: categorized_children, dtype: int64


Разобьем массив по категориям в зависимости от наличия долгов:

In [27]:
print(data['debt'].value_counts())

0    19730
1     1741
Name: debt, dtype: int64


Очевидно, что 0  соответствует состоянию без долгов, а 1 - с долгами. Создадим категории:

In [28]:
def create_category_debt(row):
    if row['debt'] == 1:
        return 'есть долг'
    return 'нет долга'

data['categorized_debt'] = data.apply(create_category_debt, axis=1)

print(data['categorized_debt'].value_counts())

нет долга    19730
есть долг     1741
Name: categorized_debt, dtype: int64


Теперь можно взглянуть на их пересечение, используем сводную таблицу:

In [29]:
child_dependency = data.pivot_table(
    index = 'categorized_debt',
    columns = 'categorized_children',
    values = 'debt',
    aggfunc = 'count',
    margins=True)
display(child_dependency.sort_index(ascending = False))

categorized_children,есть дети,нет детей,All
categorized_debt,,,
нет долга,6686,13044,19730
есть долг,678,1063,1741
All,7364,14107,21471


Вычислим долю должников в каждой категории, чтобы сравнить:

In [30]:
child_dependency = data.pivot_table(
    index = 'categorized_children',
    columns = 'categorized_debt',
    values = 'debt',
    aggfunc = 'count',
    margins=True)

child_dependency['debt_share'] = round(child_dependency['есть долг'] / child_dependency['All'] * 100, 2)

display(child_dependency.sort_index())

categorized_debt,есть долг,нет долга,All,debt_share
categorized_children,,,,
All,1741,19730,21471,8.11
есть дети,678,6686,7364,9.21
нет детей,1063,13044,14107,7.54


**Вывод**
Процент заемщиков с наличием просроченной задолженности по кредитному продукту действительно растет с появлением детей. Причем есть зависимость и от количества детей: так, доля должников с появлением первого ребенка увеличивается с 7,54% до 9,16%, с появлением второго - увеличивается еще на ,33% и достигает 9,49%. Далее доля несколько снижается, до 8,16% (ответственность вырастает с бременем или же родители адаптируются). В людом случае доля имеющих просроченную задолженность среди должников, имеющих детей, выше, чем бездетных, что должно найти отражении в составлении профилей рисков и кредитных рейтингов для выдачи кредита. 

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Добавим категории для холостых и семейных заемщиков:

Проверим значения в столбце 'family_status':

In [23]:
print(data['family_status'].value_counts())

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


У нас 5 категорий, указанных клиентами:
женат / замужем          12344 - женат
гражданский брак          4163 - женат (пусть и неофициально)
Не женат / не замужем     2810 - неженат
в разводе                 1195 - неженат
вдовец / вдова             959 - неженат
Так как нас интересует текущий статус и его влияние на изучаемые показатели, то, фактически, останется 2 категории.

In [24]:
def create_category_marriage(row):
    if ((row['family_status'] == 'женат / замужем') or
    (row['family_status'] == 'гражданский брак')):
        return 'семейный'
    return 'несемейный'
data['categorized_marriage'] = data.apply(create_category_marriage, axis=1)

print(data['categorized_marriage'].value_counts())

семейный      16507
несемейный     4964
Name: categorized_marriage, dtype: int64


Семейных клиентов значительно больше, проверим как обстоят дела с платежной дисциплиной:

In [31]:
marriage_dependency = data.pivot_table(
    index = 'family_status',
    columns = 'categorized_debt',
    values = 'debt',
    aggfunc = 'count',
    margins=True)

marriage_dependency['debt_share'] = round(marriage_dependency['есть долг'] / marriage_dependency['All'] * 100, 2)

display(marriage_dependency.sort_index(ascending = False))

categorized_debt,есть долг,нет долга,All,debt_share
family_status,,,,
женат / замужем,931,11413,12344,7.54
гражданский брак,388,3775,4163,9.32
вдовец / вдова,63,896,959,6.57
в разводе,85,1110,1195,7.11
Не женат / не замужем,274,2536,2810,9.75
All,1741,19730,21471,8.11


**Вывод**

Семейные клиенты чаще берут кредиты и они более дисциплинированны в погашении платежей: 8,5% клиентов с просроченным долгом в группе клиентов, у которых отсутствует муж/жена в настоящее время - против 7,99% в категории семейных клиентов.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Разобьем наш массив данных по категориям, в зависимости от уровня дохода. Сначала выясним минимальный, максимальный, средний и медианный доход:

In [26]:
print('минимальный доход = ', data['total_income'].min())
print('средний доход = ', data['total_income'].mean())
print('медианный доход = ', data['total_income'].median())
print('максимальный доход = ', data['total_income'].max())

минимальный доход =  20667.26379327158
средний доход =  165295.8877638656
медианный доход =  142594.39684740017
максимальный доход =  2265604.028722744


Если считать, что это доход в месяц, то суммы обнадеживающие, за чертой бедности нет ни одного клиента. Но так ли это? Наименование столбца с доходом - 'total_income', что предполагает, что доход аггрегирован за период. Здесь необходим запрос к составителю таблицы. Так как у нас такой возможности нет, ограничимся предположением. Банки собирают подтвержденный доход за предыдущий период. Этот период, исходя из обычаев делового оборота, может быть от 6 месяцев до двух лет. Мое предположение, что период равен 6 месяцам, так как даже в этом случае минимальное значение дохода за полгода (20667.26) угрожающе низко для выдачи положительного решения по кредиту. Ниже - ещё маловероятнее.
Для разбивки на категории обратимся к статистическим данным:
Прожиточный минимум (ПМ) на душу населения за 3 квартал 2021 года составляет 11 970 рубля в месяц.
Нищие – душевой доход до 1 ПМ/чел. 
Бедные - душевой доход от 1 до 3 ПМ/чел.
Выше бедности - доход от 3 до 7 ПМ/чел. 
Средний класс – доход от 7 до 11 ПМ/чел. 
Высший класс – доход более 11 ПМ/чел.
Вычислим границы категорий для нашего анализа:


In [27]:
print('нищие - до ', 11970) # звучит нетолерантно, но при проведении настоящего исследования были проверены официальные критерии нищеты ВОЗ
print('бедные - от', 11970, 'до', 11970*3)
print('средний - от', 11970*3, 'до', 11970*7)
print('выше среднего - от', 11970*7, 'до', 11970*11)
print('богатые - от', 11970*11)

нищие - до  11970
бедные - от 11970 до 35910
средний - от 35910 до 83790
выше среднего - от 83790 до 131670
богатые - от 131670


Так как у нас доход за полгода, то для получения среднемесячного дохода будем делить значение столбца 'total_income' на 6.

In [28]:
def create_category_income(row):
    if row['total_income']/6 < 11970:
            return '0 нищий'
    if row['total_income']/6 < 35910:
        return '1 бедный'
    if row['total_income']/6 < 83790:
        return '2 средний'
    if row['total_income']/6 < 131670:
        return '3 высокий'
    return '4 богатый'
data['categorized_income'] = data.apply(create_category_income, axis=1)

print(data['categorized_income'].value_counts())

1 бедный     15616
2 средний     4019
0 нищий       1615
3 высокий      178
4 богатый       43
Name: categorized_income, dtype: int64


Распределение похоже на нормальное. Проведем анализ:

In [29]:
income_dependency = data.pivot_table(
    index = 'categorized_income',
    columns = 'categorized_debt',
    values = 'debt',
    aggfunc = 'count',
    margins=True)

income_dependency['debt_share'] = round(income_dependency['есть долг'] / income_dependency['All'] * 100, 2)

display(income_dependency.sort_index(ascending = True))

categorized_debt,есть долг,нет долга,All,debt_share
categorized_income,,,,
0 нищий,110,1505,1615,6.81
1 бедный,1333,14283,15616,8.54
2 средний,284,3735,4019,7.07
3 высокий,10,168,178,5.62
4 богатый,4,39,43,9.30
All,1741,19730,21471,8.11


**Вывод**

Самые дисциплинированные плательщики - с высоким доходом (определения даются в соответствии с критериями разбиения настоящего исследования, см. текст выше), 5,62% недисциплинированных плательщиков от общего числа клиентов в категории. На втором месте - клиенты с доходом ниже прожиточного минимума, их показатель - 6,81% недисциплинированных плательщиков от общего числа клиентов в категории. На третьем месте - Клиенты со средним доходом - 7,07% недисциплинированных плательщиков от общего числа клиентов в категории. И с чувствительным отрывом идут 2 наиболее недисциплитнированных категории - бедные и богатые, 8,54% и 9,3% недисциплинированных плательщиков от общего числа клиентов в категории соответственно. Прослеживаются различия в группах Процент неплательщиков не зависит от увеличения дохода линейно,  прогрессивно или регрессивно. Однако, полученные данные помогут созданию рейтинга клиентов для одобрения выдачи кредита, с обязательным пересмотром по мере получения новых данных, но не реже, чем 1 раз в год.

**- Как разные цели кредита влияют на его возврат в срок? **

Категоризации по целям кредита мы провели ранее, проведем анализ в рамках поставленного вопроса:

In [30]:
purpose_dependency = data.pivot_table(
    index = 'categorized_purpose',
    columns = 'categorized_debt',
    values = 'debt',
    aggfunc = 'count',
    margins=True)

purpose_dependency['debt_share'] = round(purpose_dependency['есть долг'] / purpose_dependency['All'] * 100, 2)

display(purpose_dependency.sort_index(ascending = False))

categorized_debt,есть долг,нет долга,All,debt_share
categorized_purpose,,,,
свадьба,186,2149,2335,7.97
образование,370,3644,4014,9.22
недвижимость,782,10032,10814,7.23
автомобиль,403,3905,4308,9.35
All,1741,19730,21471,8.11


**Вывод**
Мы видим зависимость доли неплательщиков от категории цели, на которую получен кредит. Самые дисциплинированные плательщики - те, кто взял кредит на недвижимость. 7,97% недисциплинированных плательщиков от общего числа клиентов в категории. Это объяснимо, поскольку недвижимость обычно выступает залогом, и мотивация для плательщика максимальная. На втором месте - свадьба, 7,97, что несколько неожиданно, исходная гипотеза предполагала следующим пунктом категорию взявших кредит на автомобиль. Между тем, и взявшие кредит на автомобиль, и взявшие кредит на образование - наименее дисциплинированы, 9,35% и  9,22% недисциплинированных плательщиков от общего числа клиентов в категории соответственно.


## Шаг 4. Общий вывод

Мы провели исследование для кредитного отдела банка  для определения корреляции возврата кредита в срок и следующих показателей:
- наличия детей у заемщика;
- семейного положения заемщика;
- уровнем дохода заемщика;
- цели кредита.

**Зависимость между наличием детей и возвратом кредита в срок**
Мы установили, что процент заемщиков с наличием просроченной задолженности по кредитному продукту действительно растет с появлением детей. Причем есть зависимость не только от наличия, но и от количества детей: так, доля должников с появлением первого ребенка увеличивается с 7,54% до 9,16%, с появлением второго - увеличивается еще на ,33% и достигает 9,49%. Далее доля несколько снижается, до 8,16% (ответственность вырастает с бременем или же родители адаптируются). В любом случае, доля имеющих просроченную задолженность среди должников, имеющих детей, выше, чем бездетных, что должно найти отражении в составлении профилей рисков и кредитных рейтингов для выдачи кредита.
**Зависимость между семейным положением заемщика и возвратом кредита в срок**
Семейные клиенты чаще берут кредиты, и они более дисциплинированны в погашении платежей: 8,5% клиентов с просроченным долгом в группе клиентов, у которых отсутствует муж/жена в настоящее время - против 7,99% в категории семейных клиентов. Рекомендуем отразить выявленную зависимость при составлении профилей рисков и кредитных рейтингов для выдачи кредита
**зЗависимость между уровнем дохода и возвратом кредита в срок**
Самые дисциплинированные плательщики - с высоким доходом (определения даются в соответствии с критериями разбиения настоящего исследования, см. текст выше), 5,62% недисциплинированных плательщиков от общего числа клиентов в категории. На втором месте - клиенты с доходом ниже прожиточного минимума, их показатель - 6,81% недисциплинированных плательщиков от общего числа клиентов в категории. На третьем месте - Клиенты со средним доходом - 7,07% недисциплинированных плательщиков от общего числа клиентов в категории. И с чувствительным отрывом идут 2 наиболее недисциплинированных категории - бедные и богатые, 8,54% и 9,3% недисциплинированных плательщиков от общего числа клиентов в категории соответственно. Прослеживаются выраженные различия в категориях, выделенных по уровню дохода заемщика. Процент неплательщиков не зависит от увеличения дохода линейно, прогрессивно или регрессивно. 
**Цели кредита влияют на его возврат в срок**
Мы видим зависимость доли неплательщиков от категории цели, на которую получен кредит. Самые дисциплинированные плательщики - те, кто взял кредит на недвижимость. 7,97% недисциплинированных плательщиков от общего числа клиентов в категории. Это объяснимо, поскольку недвижимость обычно выступает залогом, и мотивация для плательщика максимальная. На втором месте - свадьба, 7,97%. Между тем, и взявшие кредит на автомобиль, и взявшие кредит на образование - наименее дисциплинированы, 9,35% и 9,22% недисциплинированных плательщиков от общего числа клиентов в категории соответственно.

Мы рекомендуем учитывать каждый из проанализированных параметров при построении модели кредитного скоринга. Данные в систему скоринга, которая оценивает способность потенциального заёмщика вернуть кредит банку необходимо перевести через систему баллов, присуждаемых плательщику в зависимости от наличия детей, семейного положения, доходов и цели получения кредита.
Полученные данные достаточны для созданию рейтинга клиентов в  целях одобрения выдачи кредита. Возможна дальнейшая детализация и параметрический анализ. Обязательным является пересмотр рейтинга по мере получения новых данных, но не реже, чем 1 раз в год.


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.